In [1]:
from selenium import webdriver 
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By
# 명시적 대기를 위해 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
main_url = 'http://tour.interpark.com/'
keyword  = input("키워드 입력: ")
driver = webdriver.Chrome(executable_path='C:/python_edu/program/chromedriver.exe')
driver.get(main_url)
# 검색창을 찾아서 검색어 입력
# id : SearchGNBText
driver.find_element_by_id('SearchGNBText').send_keys(keyword)
driver.find_element_by_css_selector('button.search-btn').click()
time.sleep(5)
driver.find_element_by_css_selector('.oTravelBox>.boxList>.moreBtnWrap>.moreBtn').click()
time.sleep(5)
totalCount = driver.find_element_by_css_selector('#totalAllCnt')
print("검색 결과:", totalCount.text)
totalCount = int(totalCount.text.replace(",",""))
totalPage =  (totalCount//10)+1
totalPage

In [2]:
# 상품 정보를 담는 클레스
class TourInfo:
    # 맴버변수 (실제 컬럼보다는 작게 세팅했음)
    title = ''
    price = ''
    info  = ''
    link  = ''
    img   = ''
    contents = ''
    # 생성자
    def __init__(self, title, price, info, link, img, contents=None ):
        self.title = title
        self.price = price
        self.info  = info
        self.link  = link
        self.img   = img
        self.contents = contents
        
# 상품 정보를 담는 리스트 (TourInfo 리스트)
tour_list = []

In [ ]:
for page in range(1, 3):
    try:
        # 자바스크립트 구동하기
        driver.execute_script("searchModule.SetCategoryList(%s, '')" % page)
        time.sleep(2)
        print("%s 페이지 이동" % page)
        
        boxItems = driver.find_elements_by_css_selector('.oTravelBox>.boxList>li')
        # 상품 하나 하나 접근
        for li in boxItems:
            img = li.find_element_by_css_selector('img').get_attribute('src') 
            link = li.find_element_by_css_selector('a').get_attribute('onclick')
            title = li.find_element_by_css_selector('h5.proTit').text 
            comment = li.find_element_by_css_selector('.proSub').text 
            price =   li.find_element_by_css_selector('.proPrice').text 
            info = ''
            for proinfo in li.find_elements_by_css_selector('.info-row .proInfo'):
                info+=proinfo.text+"\n"
            
            print("title:%s\nlink:%s\nimg:%s\ncomment:%s\nprice:%s\ninfo:%s" % (title, link, img, comment, price, info))
            print('='*100)
            obj = TourInfo(  
                title,
                price,
                info,
                link,
                img
            )
            tour_list.append( obj )
    except Exception as e1:
        print( '오류', e1 )

In [ ]:
print(len(tour_list) )

In [ ]:
tour_list[0]

In [ ]:
from selenium.webdriver.common.alert import Alert
for tour in tour_list:
    arr = tour.link.split(',')
    if arr:
        # 대체
        link = arr[0].replace('searchModule.OnClickDetail(','')
        # 슬라이싱 => 앞에 ', 뒤에 ' 제거
        detail_url = link[1:-1]
        try:
            # 상세 페이지 이동 : URL 값이 완성된 형태인지 확인 (http~)
            driver.get( detail_url )
            time.sleep(2)
            driver.execute_script("MovingPoint('CM')")
            time.sleep(2)

            comment_count = driver.find_element_by_css_selector('#hddGoodsEvalCnt').get_attribute("value")
            print(comment_count)
            comment_count = int(comment_count)
           
            tits = driver.find_elements_by_css_selector('div.article strong.tit')
            grades = driver.find_elements_by_css_selector('div.article span.grade')
            soup = BeautifulSoup( driver.page_source, 'html.parser')
            txt = soup.select("div.article div.cont")


            for i in range(0, len(tits)):
                ctitle = tits[i].text
                cgrade = grades[i].text
                ctxts = txt[i].select("p")
                ctx=""
                for i in range(0, len(ctxts)):
                     ctx+=str(ctxts[i]).replace("<p>","").replace("</p>","")
                print("title:%s\nrank:%s\nctxts:%s" %(ctitle,cgrade,ctx))
            
            
        except Exception as e:
            print(e)
            Alert(driver).accept()
            timer.sleep(3)